In [2]:
!pip install pandas

    100% |################################| 9.5MB 184kB/s  eta 0:00:01


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.pandas.utils import require_minimum_pandas_version, require_minimum_pyarrow_version

spark = SparkSession \
        .builder \
        .appName("Python Arrow-in-Spark example") \
        .getOrCreate()
# builder.master("local[1]") \ 
sc = spark.sparkContext

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [109]:
spark.stop()

In [6]:
spark

In [9]:
import numpy as np  # type: ignore[import]
import pandas as pd  # type: ignore[import]

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

# Generate a Pandas DataFrame
pdf = pd.DataFrame(np.random.rand(100, 3))

# Create a Spark DataFrame from a Pandas DataFrame using Arrow
df = spark.createDataFrame(pdf)

# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = df.select("*").toPandas()

print("Pandas DataFrame result statistics:\n%s\n" % str(result_pdf.describe()))


/usr/local/lib/python3.6/dist-packages/pyspark/sql/pandas/conversion.py:329: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 1.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)
/usr/local/lib/python3.6/dist-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 1.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


Pandas DataFrame result statistics:
                0           1           2
count  100.000000  100.000000  100.000000
mean     0.493640    0.507350    0.530803
std      0.276708    0.293061    0.279217
min      0.028506    0.022717    0.002653
25%      0.268534    0.270429    0.286999
50%      0.497026    0.499115    0.545593
75%      0.704514    0.791336    0.755214
max      0.991715    0.984116    0.979381



In [11]:
pdf.describe()

,0,1,2
count,100.000000,100.000000,100.000000
mean,0.493640,0.507350,0.530803
std,0.276708,0.293061,0.279217
min,0.028506,0.022717,0.002653
25%,0.268534,0.270429,0.286999
50%,0.497026,0.499115,0.545593
75%,0.704514,0.791336,0.755214
max,0.991715,0.984116,0.979381


In [13]:
df = spark.read.json("data/people.json")
# Displays the content of the DataFrame to stdout
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [14]:
df.select("age").show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [16]:
df.select(["age", "name"]).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [22]:
df.select([df["age"]+1, "age"]).show()

+---------+----+
|(age + 1)| age|
+---------+----+
|     null|null|
|       31|  30|
|       20|  19|
+---------+----+



In [28]:
df.select(df["age"]).show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [29]:
df.createOrReplaceTempView("people2")

In [33]:
sqlDF = spark.sql("SELECT age FROM people2")
sqlDF.show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [38]:
lines = spark.sparkContext.textFile("data/people.txt")
lines

data/people.txt MapPartitionsRDD[58] at textFile at NativeMethodAccessorImpl.java:0

In [53]:
from __future__ import print_function
lines.foreach(print)

In [47]:
lines.collect()

['Michael, 29', 'Andy, 30', 'Justin, 19']

In [54]:
for x in lines.collect():
    print(x)

Michael, 29
Andy, 30
Justin, 19


In [56]:
print(lines.take(2))

['Michael, 29', 'Andy, 30']


In [59]:
lines.take(10)

['Michael, 29', 'Andy, 30', 'Justin, 19']

In [61]:
data = lines.flatMap(lambda x: x.split(' '))
map = data.map(lambda x: (x, 1))
mapreduce = map.reduceByKey(lambda x,y: x+y)
result = mapreduce.collect()
result

[('29', 1),
 ('Andy,', 1),
 ('Justin,', 1),
 ('19', 1),
 ('Michael,', 1),
 ('30', 1)]

In [64]:
data = lines.flatMap(lambda x: x.split(' '))
data.take(10)

['Michael,', '29', 'Andy,', '30', 'Justin,', '19']

In [65]:
rdd = spark.sparkContext.parallelize(range(1, 4)).map(lambda x: (x, "a" * x))
rdd

PythonRDD[81] at RDD at PythonRDD.scala:53

In [66]:
rdd.saveAsSequenceFile("data/seq_file")

In [69]:
sorted(spark.sparkContext.sequenceFile("data/seq_file").collect())

[(1, 'a'), (2, 'aa'), (3, 'aaa')]

In [77]:
lines = sc.textFile("data/people.txt")
lineLengths = lines.map(lambda s: len(s))
totalLength = lineLengths.reduce(lambda a, b: a + b)
totalLength

29

In [79]:
lineLengths.collect()

[11, 8, 10]

In [78]:
lineLengths.persist()

PythonRDD[106] at RDD at PythonRDD.scala:53

In [82]:
lineLengths5 = lineLengths.map(lambda x: x+5)

In [83]:
lineLengths5.collect()

[16, 13, 15]

---  

In [101]:
data = np.arange(1, 100)

counter = 0
rdd = sc.parallelize(data)

# Wrong: Don't do this!!
def increment_counter(x):
    global counter
    counter += x
    print(counter)

rdd.foreach(increment_counter)

print("Counter value: ", counter)

Counter value:  0


In [8]:
# Working with Key-Value Pairs
#lines = sc.textFile("data.txt")
lines = sc.parallelize(['a', 'b', 'a', 'a', 'a', 'b', 'c', 'a', 'b', 'a', 'a', 'a', 'b', 'c', 'a', 'b', 'a', 'a', 'a', 'b', 'c'])
pairs = lines.map(lambda s: (s, 1))
counts = pairs.reduceByKey(lambda a, b: a + b)
counts.take(10)

[('a', 12), ('b', 6), ('c', 3)]

In [25]:
cnts = pairs.groupByKey()
# cnts = cnts.collect()
lst = cnts.take(10)

In [24]:
pairs.groupByKey().map(lambda x : (x[0], list(x[1]))).collect()

[('a', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 ('b', [1, 1, 1, 1, 1, 1]),
 ('c', [1, 1, 1])]

In [30]:
for i in lst[1][1] : 
    print(i)

1
1
1
1
1
1


In [31]:
pairs.groupByKey().mapValues(list).collect()

[('a', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 ('b', [1, 1, 1, 1, 1, 1]),
 ('c', [1, 1, 1])]

In [33]:
pairs.groupByKey().map(list).collect()

[['a', <pyspark.resultiterable.ResultIterable at 0x7f9e39666240>],
 ['b', <pyspark.resultiterable.ResultIterable at 0x7f9e396cf240>],
 ['c', <pyspark.resultiterable.ResultIterable at 0x7f9e396cf048>]]

In [34]:
schemaPeople = spark.createDataFrame(pairs)
schemaPeople.createOrReplaceTempView("pairs")

In [35]:
teenagers = spark.sql("SELECT * FROM pairs")
teenagers

DataFrame[_1: string, _2: bigint]

In [36]:
teenagers.collect()

[Row(_1='a', _2=1),
 Row(_1='b', _2=1),
 Row(_1='a', _2=1),
 Row(_1='a', _2=1),
 Row(_1='a', _2=1),
 Row(_1='b', _2=1),
 Row(_1='c', _2=1),
 Row(_1='a', _2=1),
 Row(_1='b', _2=1),
 Row(_1='a', _2=1),
 Row(_1='a', _2=1),
 Row(_1='a', _2=1),
 Row(_1='b', _2=1),
 Row(_1='c', _2=1),
 Row(_1='a', _2=1),
 Row(_1='b', _2=1),
 Row(_1='a', _2=1),
 Row(_1='a', _2=1),
 Row(_1='a', _2=1),
 Row(_1='b', _2=1),
 Row(_1='c', _2=1)]

In [38]:
teenNames = teenagers.rdd.map(lambda p: "Name: " + p._1).collect()
teenNames

['Name: a',
 'Name: b',
 'Name: a',
 'Name: a',
 'Name: a',
 'Name: b',
 'Name: c',
 'Name: a',
 'Name: b',
 'Name: a',
 'Name: a',
 'Name: a',
 'Name: b',
 'Name: c',
 'Name: a',
 'Name: b',
 'Name: a',
 'Name: a',
 'Name: a',
 'Name: b',
 'Name: c']

In [39]:
for name in teenNames:
    print(name)

Name: a
Name: b
Name: a
Name: a
Name: a
Name: b
Name: c
Name: a
Name: b
Name: a
Name: a
Name: a
Name: b
Name: c
Name: a
Name: b
Name: a
Name: a
Name: a
Name: b
Name: c


In [44]:
from pyspark.sql.types import StringType, StructType, StructField

schemaString = "name count"
fields = [StructField(fname, StringType(), True) for fname in schemaString.split()]
schema = StructType(fields)

schemaPeople = spark.createDataFrame(pairs, schema)
schemaPeople

DataFrame[name: string, count: string]

In [46]:
schemaPeople.createOrReplaceTempView("pairs_scheme")
queried = spark.sql("select * from pairs_scheme")
queried.collect()

[Row(name='a', count='1'),
 Row(name='b', count='1'),
 Row(name='a', count='1'),
 Row(name='a', count='1'),
 Row(name='a', count='1'),
 Row(name='b', count='1'),
 Row(name='c', count='1'),
 Row(name='a', count='1'),
 Row(name='b', count='1'),
 Row(name='a', count='1'),
 Row(name='a', count='1'),
 Row(name='a', count='1'),
 Row(name='b', count='1'),
 Row(name='c', count='1'),
 Row(name='a', count='1'),
 Row(name='b', count='1'),
 Row(name='a', count='1'),
 Row(name='a', count='1'),
 Row(name='a', count='1'),
 Row(name='b', count='1'),
 Row(name='c', count='1')]

In [48]:
print(queried)
queried.show()

DataFrame[name: string, count: string]
+----+-----+
|name|count|
+----+-----+
|   a|    1|
|   b|    1|
|   a|    1|
|   a|    1|
|   a|    1|
|   b|    1|
|   c|    1|
|   a|    1|
|   b|    1|
|   a|    1|
|   a|    1|
|   a|    1|
|   b|    1|
|   c|    1|
|   a|    1|
|   b|    1|
|   a|    1|
|   a|    1|
|   a|    1|
|   b|    1|
+----+-----+
only showing top 20 rows



In [63]:
queried

DataFrame[name: string, count: string]

In [69]:
queried.rdd.map(lambda x: x.name + x['count']).collect()

['a1',
 'b1',
 'a1',
 'a1',
 'a1',
 'b1',
 'c1',
 'a1',
 'b1',
 'a1',
 'a1',
 'a1',
 'b1',
 'c1',
 'a1',
 'b1',
 'a1',
 'a1',
 'a1',
 'b1',
 'c1']

In [70]:
dir(queried)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collect_as_arrow',
 '_jcols',
 '_jdf',
 '_jmap',
 '_jseq',
 '_lazy_rdd',
 '_repr_html_',
 '_sc',
 '_schema',
 '_sort_cols',
 '_support_repr_html',
 '_to_corrected_pandas_type',
 'agg',
 'alias',
 'approxQuantile',
 'cache',
 'checkpoint',
 'coalesce',
 'colRegex',
 'collect',
 'columns',
 'corr',
 'count',
 'cov',
 'createGlobalTempView',
 'createOrReplaceGlobalTempView',
 'createOrReplaceTempView',
 'createTempView',
 'crossJoin',
 'crosstab',
 'cube',
 'describe',
 'distinct',
 'drop',
 'dropDuplicates',
 'drop_duplicates',
 'dropna',
 'dtypes',
 'exceptAll',
 'explain',
 'fillna',
 'fi